# NeqSim Risk Simulation & Equipment Failure Analysis

Complete guide to production risk analysis, equipment failure simulation, and degraded operation optimization.

## Features
1. **Production Impact Analysis** - Quantify production loss from equipment failures
2. **Degraded Operation Optimization** - Find optimal operating points during outages
3. **Monte Carlo Risk Simulation** - Simulate production availability over time
4. **Equipment Reliability Database** - Access OREDA-based failure data
5. **Recovery Planning** - Generate step-by-step recovery procedures

In [2]:
# Import NeqSim - Direct Java Access via jneqsim
from neqsim import jneqsim
import json

# Import Java classes through the jneqsim gateway
SystemSrkEos = jneqsim.thermo.system.SystemSrkEos
ProcessSystem = jneqsim.process.processmodel.ProcessSystem
Stream = jneqsim.process.equipment.stream.Stream
Separator = jneqsim.process.equipment.separator.Separator
Compressor = jneqsim.process.equipment.compressor.Compressor
Cooler = jneqsim.process.equipment.heatexchanger.Cooler
ProductionImpactAnalyzer = jneqsim.process.util.optimizer.ProductionImpactAnalyzer
DegradedOperationOptimizer = jneqsim.process.util.optimizer.DegradedOperationOptimizer
OperationalRiskSimulator = jneqsim.process.safety.risk.OperationalRiskSimulator
ReliabilityDataSource = jneqsim.process.equipment.failure.ReliabilityDataSource
EquipmentFailureMode = jneqsim.process.equipment.failure.EquipmentFailureMode

print("NeqSim Risk Analysis Framework loaded!")

NeqSim Risk Analysis Framework loaded!


## 1. Create Test Process

Gas processing facility with inlet separator and compression.

In [ ]:
# Create fluid
fluid = SystemSrkEos(280.0, 50.0)
fluid.addComponent("methane", 0.85)
fluid.addComponent("ethane", 0.10)
fluid.addComponent("propane", 0.05)
fluid.setMixingRule("classic")

# Build process
process = ProcessSystem()

feed = Stream("Well Feed", fluid)
feed.setFlowRate(10000.0, "kg/hr")
feed.setTemperature(35.0, "C")
feed.setPressure(30.0, "bara")
process.add(feed)

separator = Separator("Inlet Separator", feed)
process.add(separator)

compressor = Compressor("Export Compressor", separator.getGasOutStream())
compressor.setOutletPressure(100.0, "bara")
process.add(compressor)

cooler = Cooler("Export Cooler", compressor.getOutletStream())
cooler.setOutTemperature(30.0, "C")
process.add(cooler)

export = Stream("Export Gas", cooler.getOutletStream())
process.add(export)

process.run()
print(f"Baseline Production: {export.getFlowRate('kg/hr'):.0f} kg/hr")

## 2. Production Impact Analysis

Analyze how equipment failures affect production.

In [ ]:
# Create analyzer
analyzer = ProductionImpactAnalyzer(process)
analyzer.setFeedStreamName("Well Feed")
analyzer.setProductStreamName("Export Gas")

# Analyze compressor failure
impact = analyzer.analyzeFailureImpact("Export Compressor")

print("=== Compressor Failure Impact ===")
print(f"Baseline: {impact.getBaselineProduction():.0f} kg/hr")
print(f"With Failure: {impact.getProductionWithFailure():.0f} kg/hr")
print(f"Loss: {impact.getProductionLoss():.0f} kg/hr ({impact.getProductionLossPercent():.1f}%)")
print(f"Action: {impact.getRecommendedAction()}")

In [ ]:
# Rank equipment by criticality
criticality = dict(analyzer.rankEquipmentByCriticality())
print("\n=== Equipment Criticality ===")
for equip, loss_pct in sorted(criticality.items(), key=lambda x: -x[1]):
    print(f"  {equip}: {loss_pct:.1f}% production loss")

## 3. Degraded Operation Optimization

Find optimal setpoints when equipment is down.

In [ ]:
# Create optimizer
optimizer = DegradedOperationOptimizer(process)
optimizer.setFeedStreamName("Well Feed")
optimizer.setProductStreamName("Export Gas")

# Optimize with compressor down
result = optimizer.optimizeWithEquipmentDown("Export Compressor")

print("=== Optimized Degraded Operation ===")
print(f"Capacity Factor: {result.getCapacityFactor()*100:.1f}%")
print(f"Optimal Flow: {result.getOptimalFlowRate():.0f} kg/hr")
print(f"Production: {result.getOptimalProduction():.0f} kg/hr")

In [ ]:
# Generate recovery plan
plan = optimizer.createRecoveryPlan("Export Compressor")
print(plan.toString())

## 4. Monte Carlo Risk Simulation

Simulate production availability over time.

In [ ]:
# Access reliability database
reliability = ReliabilityDataSource.getInstance()

# Get compressor data
comp_rel = reliability.getReliabilityData("Compressor", "Centrifugal")
print("=== Centrifugal Compressor Reliability ===")
print(f"MTBF: {comp_rel.getMtbf():.0f} hours")
print(f"MTTR: {comp_rel.getMttr():.0f} hours")
print(f"Availability: {comp_rel.getAvailability()*100:.2f}%")

In [ ]:
# Create risk simulator
risk_sim = OperationalRiskSimulator(process)
risk_sim.setFeedStreamName("Well Feed")
risk_sim.setProductStreamName("Export Gas")
risk_sim.setRandomSeed(42)

# Add equipment reliability (MTBF, MTTR in hours)
risk_sim.addEquipmentMtbf("Export Compressor", 25000, 72)
risk_sim.addEquipmentMtbf("Export Cooler", 50000, 24)
risk_sim.addEquipmentMtbf("Inlet Separator", 250000, 48)

# Run simulation (1000 iterations, 1 year)
mc_result = risk_sim.runSimulation(1000, 365)

print("=== Monte Carlo Results (1 Year) ===")
print(f"Availability: {mc_result.getMeanAvailability():.2f}%")
print(f"Efficiency: {mc_result.getProductionEfficiency():.2f}%")
print(f"Mean Failures: {mc_result.getMeanFailureCount():.1f}")
print(f"Mean Downtime: {mc_result.getMeanDowntimeHours():.1f} hrs")
print(f"\nProduction (P10/P50/P90):")
print(f"  {mc_result.getP10Production()/1e6:.1f} / {mc_result.getP50Production()/1e6:.1f} / {mc_result.getP90Production()/1e6:.1f} tonnes")

In [ ]:
# Full JSON report
print(json.dumps(json.loads(mc_result.toJson()), indent=2))

## Summary

NeqSim provides comprehensive tools for:
- Quantifying production impact of equipment failures
- Optimizing plant operation during outages
- Simulating long-term production availability
- Risk-based maintenance planning